# **Minecraft Server in Google Colab!**
**by tester247#2636**

Hover over each cell and click run on the left side

Follow through setup if its your first time.. if not skip part 1

If you have allready uploaded all your server files in "Minecraft" folder in Gdrive then skip part 1

In [ ]:
#If you already have all your server files then put them in a folder called "MineCraft"
#Make sure the main jar file (spigot, vanilla, forge,etc) is named server.jar
#zip that folder..name it MineCraft.zip and upload it to the main directory of your Gdrive
#run this sell to unzip the contents and you can now skip part 1

from google.colab import drive
drive.mount('/content/drive')
%cd "/content/drive/My Drive/
!unzip MineCraft.zip

#Part 1: Setup (First Timers)

### Step 1: Setup reverse proxy to host your server

First go to https://dashboard.ngrok.com/ and create a account, then continue.

### Step 2: Setting up all the server files

In [ ]:
#Attaching your google drive to store the server files
drive.mount('/content/drive')

# Create the directory which will be used for the server
#This will create a folder called "MineCraft" in your Gdrive 
#make sure there isn't already a folder with the same name

!mkdir "/content/drive/My Drive/MineCraft"

#Agree to EULA
%cd "/content/drive/My Drive/MineCraft"
!echo eula=true > eula.txt

#UPLOADING your server.properties file
#in your properties file make sure the server ip is blank, server port is 25565, and online mode is set to false
#also if using your own world then make sure the level-name entry matches excactly with your world folder name
from google.colab import files
source = files.upload()

In [ ]:
#Optional
#If you have your own world whose name you have already entered in the properties file
#Then..select the world folder (all three if nether end over are seprate) and zip them
#Rename to world.zip and run this cell to upload and unzip
%cd "/content/drive/My Drive/MineCraft"
from google.colab import files
source = files.upload()
!unzip world.zip

In [ ]:
#Upload your plugins 
!mkdir "/content/drive/My Drive/MineCraft/plugins"
%cd "/content/drive/My Drive/MineCraft/plugins"
from google.colab import files
source = files.upload()

You can always come back to this step to add more plugins
to remove them go the the MineCraft folder in Gdrive and delete the jar files

 # Part 2: Start Your Server

In [ ]:
!pip -q install pyngrok
import os
import re
from pyngrok import conf, ngrok

#Set your timezone for correct time in logs/console
!rm /etc/localtime
!ln -s /usr/share/zoneinfo/Asia/Kolkata /etc/localtime
!date

#You can find your authtoken at https://dashboard.ngrok.com/auth/your-authtoken
NGROK_AUTH = "" ##Paste token here between the quotes

#Making sure google drive is mounted
from google.colab import drive
drive.mount('/content/drive')
%cd "/content/drive/MyDrive/MineCraft"

#Updating JAVA
# Update the package lists
!sudo apt update &>/dev/null && echo "apt cache successfully updated" || echo "apt cache update failed, you might receive stale packages"
# Install OpenJDK 15
!wget -qO - https://adoptopenjdk.jfrog.io/adoptopenjdk/api/gpg/key/public | sudo apt-key add -
!sudo add-apt-repository --yes https://adoptopenjdk.jfrog.io/adoptopenjdk/deb/ &>/dev/null || echo "Failed to add repo. Still can be ignored if openjdk15 gets installed."
!sudo apt-get install adoptopenjdk-15-hotspot-jre &>/dev/null && echo "Yay! Openjdk15 has been successfully installed." || echo "Failed to install OpenJdk15."
#Perform java version check
java_ver = !java -version 2>&1 | awk -F[\"\.] -v OFS=. 'NR==1{print $2}'
if java_ver[0] == "15" :
  print("Openjdk15 is working correctly, you are good to go.")
else:
  print("Openjdk15 doesn't seems to be installed or isn't working, falling back to java", java_ver[0], ". You might experience reduced performance.")

#Downloading the server jar
# Replace "1.16.3" with your desired server version.
version = '1.16.3'

from google.colab import drive
import requests

#Removing the server jar file if its already there (as sometimes it doesn't run unless downloaded everytime)
!rm "/content/drive/My Drive/MineCraft/server.jar"

print('Downloading to Google Drive...')
 #if using some other version (vanilla, paper, forge, etc.. then put direct url under single quotes below )
url = 'https://cdn.getbukkit.org/spigot/spigot-' + version + '.jar' 

r = requests.get(url)

if r.status_code is 200:
  with open('/content/drive/My Drive/MineCraft/server.jar', 'wb') as f:
    f.write(r.content)
  print('Done!')
else:
  print('Error '+ str(r.status_code) + '! Most likely you entered a unsupported version. Try running the code again if you think that shouldn\'t have happened.')


! ngrok authtoken  $NGROK_AUTH # login to ngrok


# Connect to ngrok
#by default us region shoud work the best.. if any connection issues u can change it to
#eu,ap,au,sa,jp or in
get_ipython().system_raw('ngrok tcp -region us 25565 &') 

#Get public IP 
!apt install jq
print("THIS IS YOUR SERVER URL")
!curl --silent http://127.0.0.1:4040/api/tunnels | jq '.tunnels[0].public_url'

#Starting server (ive set min and max 10GB memory allocation )
print('Starting server...')
!java -Xmx10G -Xms10G -XX:+UseG1GC -XX:+ParallelRefProcEnabled -XX:MaxGCPauseMillis=200 -XX:+UnlockExperimentalVMOptions -XX:+DisableExplicitGC -XX:+AlwaysPreTouch -XX:G1NewSizePercent=30 -XX:G1MaxNewSizePercent=40 -XX:G1HeapRegionSize=8M -XX:G1ReservePercent=20 -XX:G1HeapWastePercent=5 -XX:G1MixedGCCountTarget=4 -XX:InitiatingHeapOccupancyPercent=15 -XX:G1MixedGCLiveThresholdPercent=90 -XX:G1RSetUpdatingPauseTimePercent=5 -XX:SurvivorRatio=32 -XX:+PerfDisableSharedMem -XX:MaxTenuringThreshold=1 -Dusing.aikars.flags=https://mcflags.emc.gs -Daikars.new.flags=true -jar server.jar nogui


For restarting server stop the above cell and run this

In [ ]:
from google.colab import drive
import requests

#Removing the server jar file if its already there (as sometimes it doesn't run unless downloaded everytime)
!rm "/content/drive/My Drive/MineCraft/server.jar"

print('Downloading to Google Drive...')
 #if using some other version (vanilla, paper, forge, etc.. then put direct url under single quotes below )
url = 'https://cdn.getbukkit.org/spigot/spigot-' + version + '.jar' 

r = requests.get(url)

if r.status_code is 200:
  with open('/content/drive/My Drive/MineCraft/server.jar', 'wb') as f:
    f.write(r.content)
  print('Done!')
else:
  print('Error '+ str(r.status_code) + '! Most likely you entered a unsupported version. Try running the code again if you think that shouldn\'t have happened.')
print('Starting server...')
!java -Xmx10G -Xms10G -XX:+UseG1GC -XX:+ParallelRefProcEnabled -XX:MaxGCPauseMillis=200 -XX:+UnlockExperimentalVMOptions -XX:+DisableExplicitGC -XX:+AlwaysPreTouch -XX:G1NewSizePercent=30 -XX:G1MaxNewSizePercent=40 -XX:G1HeapRegionSize=8M -XX:G1ReservePercent=20 -XX:G1HeapWastePercent=5 -XX:G1MixedGCCountTarget=4 -XX:InitiatingHeapOccupancyPercent=15 -XX:G1MixedGCLiveThresholdPercent=90 -XX:G1RSetUpdatingPauseTimePercent=5 -XX:SurvivorRatio=32 -XX:+PerfDisableSharedMem -XX:MaxTenuringThreshold=1 -Dusing.aikars.flags=https://mcflags.emc.gs -Daikars.new.flags=true -jar server.jar nogui

Stop ngrok tunnel after done

In [ ]:
!kill $(ps aux | grep 'ngrok' | awk '{print $2}') 

# Backing-up the World

go to google drive and download the world folder duh